# GRT Jacobian Tests Unified

In this Notebook we will test the calculation of the Jacobians.

In [54]:
from gravray import *
from gravray.util import *
from gravray.sampling import *
from gravray.spice import *
from gravray.orbit import *
from gravray.stats import *
Spice.loadKernels()

## Single ray

In [56]:
#Parameters
body="EARTH"
lon=61.1*Angle.Deg #deg
lat=54.8*Angle.Deg #deg
alt=23.3*Const.km #m
#Ray
A=101.1*Angle.Deg
h=15.9*Angle.Deg
v=-18.6*Const.kms

#Location
earth=Body(body)
chelyabinsk=Location(earth,lon,lat,alt)
ray=GrtRay(chelyabinsk,A,h,v)
#Time
tdb=Spice.str2t("02/15/2013 03:20:34 UTC")
#Propagate ray
ray.updateRay(tdb)
ray.propagateRay()
detJ=ray.calcJacobianDeterminant()
print("Terminal orbit elements:",Const.transformElements(ray.terminal.celements,[1/Const.au,Angle.Rad]))
print("det(Jhi) = ",detJ)

Terminal orbit elements: [  1.6400856    0.54966922   4.04158232 326.57255475 106.86339209
  21.32411541]
det(Jhi) =  -5.369756753915885e-13


## Analytical Jacobian simplified procedure

### Jxi := dXgeo/dRimp

In [26]:
ray.updateRay(tdb)
ray.calcJacobiansEcliptic()
Jxi=np.copy(ray.Jcl)
#Jxi=np.copy(ray.Jel)
print("Jxi := dXgeo/dRimp = ",Jxi)
detJxi=np.linalg.det(Jxi)
print("det(Jxi)  = ",detJxi)

Jxi := dXgeo/dRimp =  [[-3.23767769e+06 -2.52801038e+06  2.78579584e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.78729411e+06 -4.57948291e+06  5.04646045e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  3.69000654e+06  8.17144898e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.33886204e+04  1.38595563e+03 -3.66988960e-05 -9.94719861e+03
  -8.97352124e+03 -6.76780178e-01]
 [ 1.23526608e+04  1.72353048e+03  2.02588790e-05 -1.08932417e+04
  -5.90893448e+03  7.26806242e-01]
 [ 0.00000000e+00 -5.75146402e+03  0.00000000e+00  1.01185455e+04
  -1.51828986e+04  1.17137852e-01]]
det(Jxi)  =  -7.87694899143873e+21


### Jhx := dehel/dXSoI

In [27]:
#dehel/dXSoI
hel_aelements=ray.conics[2]
hel_mu=hel_aelements[-2]
hel_elements=np.array(hel_aelements[:6])
hel=KeplerianOrbit(hel_mu)
hel.setElements(hel_elements,0)
hel.calcJacobians()
Jhx=np.copy(hel.Jkc)
print("Jhs := dehel/dXsoi = ",Jhx)
detJhx=np.linalg.det(Jhx)
print("det(Jse)  = ",detJhx)

Jhs := dehel/dXsoi =  [[ 7.69123606e-01  5.30370519e+00  2.02804080e-02 -2.60663355e+07
   1.50315370e+07 -4.68979746e+06]
 [-2.86052762e-12  6.74022613e-12 -6.19266682e-13 -3.61020347e-05
   3.89708478e-05 -6.86840134e-06]
 [-4.72051920e-13  5.77152713e-14  2.80869549e-12  5.00044691e-06
  -6.11377981e-07 -2.97525168e-05]
 [-6.75629975e-12  8.26056747e-13  4.01997913e-11 -6.79136142e-07
   8.30343549e-08  4.04084072e-06]
 [ 1.82812220e-11  9.88370457e-12 -3.79025747e-11 -1.19206390e-04
  -1.09418748e-05 -2.39082960e-05]
 [-4.54876607e-12 -1.22984193e-11 -5.11784618e-13  9.53774809e-05
  -4.19220830e-05  1.68913531e-05]]
det(Jse)  =  3.1212351298525724e-35


### Resulting Jacobian, |Jhi| = |Jhx| |Jxi| 

In [29]:
detJhi=detJhx*detJxi
print("deg(Jhi) = ",detJhi)

deg(Jhi) =  -2.458580990813535e-13


## Numerical Jacobian: from Rimp to Ehel = dehel/dRimp

This is the target Jacobian, that bringing us from local impact conditions to heliocentric orbital elements

In [30]:
def Rimp2Ehel(X):
    global tdb,earth
    #Input
    lon,lat,alt,A,h,v=X
    #print("Input:",[lon*Angle.Rad,lat*Angle.Rad,alt,A*Angle.Rad,h*Angle.Rad,v])
    #Definition
    site=Location(earth,lon,lat,alt)
    ray=GrtRay(site,A,h,v)
    #Propagate
    ray.updateRay(tdb)
    ray.propagateRay()
    #Final elements
    hel=ray.terminal
    """
    helE=Const.transformElements(hel.elements,[1/Const.au,1.0])
    hel.elements=helE
    hel.calcUelements()
    #"""
    #return hel.elements
    #return hel.uelements
    #return Const.transformElements(hel.elements,[1,Angle.Rad])
    #return Const.transformElements(hel.celements,[1/Const.au,Angle.Rad])
    #return Const.transformElements(hel.celements,[1/Const.au,1])
    #print("Elements:",hel.celements)
    return hel.celements

X=np.array([lon,lat,alt,A,h,v])
dX=np.abs(X*1e-5)
e=Rimp2Ehel(X)
print("Elements:",Const.transformElements(e,[1/Const.au,Angle.Rad]))
Jhi_num=Util.computeJacobian(Rimp2Ehel,X,dX,N=6)
print("Jacobian Matrix:",Jhi_num)
detJhi_num=np.linalg.det(Jhi_num)
print("Jacobian determinant:",detJhi_num)

Elements: [  1.59290332   0.5142915    9.61015011 263.0293471  112.06897581
  21.56745495]
Jacobian Matrix: [[ 3.73033895e+11 -3.82199238e+09  2.26869968e+03 -3.00979566e+11
  -2.66588421e+11 -1.50564054e+07]
 [ 5.02475799e-01 -1.90419369e-02  1.13204267e-08 -3.49374647e-01
  -4.52549164e-01 -4.05746590e-05]
 [-7.97674958e-02  9.77049196e-02  1.94032213e-08 -1.65869641e-01
   4.58334889e-01 -2.22348874e-05]
 [-1.68924496e-04  3.66859199e-04  7.07188242e-11 -4.74918940e-04
   1.22050869e-03 -2.64693428e-08]
 [ 1.76373020e+00  4.10116245e-02 -2.47929860e-08 -1.66267278e+00
  -8.60890419e-01  1.58796326e-05]
 [-1.37320172e+00  5.02487076e-03 -2.33453028e-09  1.14581591e+00
   9.18367859e-01  4.08424352e-05]]
Jacobian determinant: 7.926907170013695e-14


In [58]:
%timeit -n 10 Util.computeJacobian(Rimp2Ehel,X,dX,N=6)

10 loops, best of 3: 130 ms per loop


## Analytical jacobians (standard method)

We want to transform a vector Rimp: (lon,lat,alt,A,h,v) into (unbounded) orbital elements Ehel: (Q, E, I, O, W, m).

This process is complex and involves the following steps and corresponding Jacobians:

- Rimp to Xgeo: Jxi = dXgeo/dRimp
- Xgeo to egeo (surface): Jex = degeosurf/dXgeo
- egeo (soi) to Xsoi: Jse = dXsoi/degeosoi
- Xsoi to ehel: Jhs = dehel/dXsoi

This Jacobians can be grouped in two sets:

- (Rimp to Xgeo) * (Xgeo to egeo): |Jxi| |Jex| = Jei = degeosurf/dRimp
- (egeo (soi) to Xsoi) * (Xsoi to ehel): |Jse| |Jhs| = Jhe = dehel/degeosoi

The full jacobian is:

- |Jei| |Jhe| = Jhi = dehel/dRimp

### Jxi

In [31]:
ray.updateRay(tdb)
ray.calcJacobiansEcliptic()
Jxi=np.copy(ray.Jel)
print("Jxi := dXgeo/dRimp = ",Jxi)
detJxi=np.linalg.det(Jxi)
print("det(Jxi)  = ",detJxi)

Jxi := dXgeo/dRimp =  [[ 9.40426962e+05  5.06402865e+06 -5.56361077e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-3.28210601e+06  2.68532317e+06  1.90825207e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.42139169e+06  2.85014065e+06  8.08732399e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.81791320e+04 -2.21170826e+03  1.06596872e-05  1.47620707e+04
   1.04478733e+04 -5.89048560e-02]
 [ 1.08066291e+03 -2.45713668e+03 -3.72024885e-05  4.30716264e+03
  -8.26275060e+03 -8.62947470e-01]
 [-4.41260292e+02 -5.20015333e+03  1.61113956e-05  9.13914959e+03
  -1.29818680e+04  5.01848465e-01]]
det(Jxi)  =  -7.876948991438786e+21


### Jex

In [32]:
geo_aelements=ray.conics[0]
geo_mu=geo_aelements[-2]
geo_elements=np.array(geo_aelements[:6])
geo=KeplerianOrbit(geo_mu)
geo.setElements(geo_elements,0)
geo.calcJacobians()
Jex=np.copy(geo.Jkc)
print("Jex := degeosurf/dXgeo = ",Jex)
detJex=np.linalg.det(Jex)
print("det(Jxi)  = ",detJex)

Jex := degeosurf/dXgeo =  [[-9.70558782e-02  3.29103912e-02  1.40207206e-01  2.06902730e+01
   2.81058311e+02 -1.64079829e+02]
 [-4.69653635e-07  3.25860132e-07  6.00854346e-07  2.17652335e-07
   5.13862859e-04 -2.39716248e-04]
 [-1.08743521e-07 -3.16087949e-08 -6.78562849e-08  1.57975953e-05
   4.59193289e-06  9.85774710e-06]
 [-9.18126413e-08 -2.66874470e-08 -5.72913651e-08 -5.12874183e-05
  -1.49078628e-05 -3.20035037e-05]
 [-3.64489966e-08 -5.27323373e-08 -2.84880306e-08  1.13379559e-05
  -4.55907651e-05 -5.91971357e-05]
 [ 8.66382884e-08  4.16969393e-07 -3.33075301e-07 -1.05727889e-04
  -6.74734917e-05  2.00865382e-04]]
det(Jxi)  =  -5.049686363246858e-26


### Composed: Jei = Jex * Jxi 

In [33]:
detJei=detJxi*detJex
print("Composed Jei := degeosurf/dRimp = ",detJei)

Composed Jei := degeosurf/dRimp =  0.00039776121906059535


### Jse

In [34]:
soi_aelements=ray.conics[1]
soi_mu=soi_aelements[-2]
soi_elements=np.array(soi_aelements[:6])
soi=KeplerianOrbit(soi_mu)
soi.setElements(soi_elements,0)
soi.calcJacobians()
Jse=np.copy(soi.Jck)
print("Jse := dXsoi/degeosoi = ",Jse)
detJse=np.linalg.det(Jse)
print("det(Jse)  = ",detJse)

Jse := dXsoi/degeosoi =  [[-2.50370564e+01 -4.98663560e+07 -5.46290125e+08  1.38340849e+09
  -8.47272668e+08 -6.48461970e+04]
 [ 7.35010731e+02  2.65497203e+07 -1.58791737e+08  4.71237698e+07
   4.44383389e+08  1.74490857e+06]
 [-3.02259191e+02  6.75462627e+07 -3.40886686e+08  0.00000000e+00
   1.15079966e+09 -7.08893119e+05]
 [-1.33193032e-04  4.74558963e+02  5.26322075e+03 -1.34913816e+04
   8.24679176e+03 -7.24791231e-04]
 [ 3.58401379e-03 -2.44966043e+02  1.52987566e+03 -5.01381447e+02
  -4.25705539e+03  2.12776344e-02]
 [-1.45605492e-03 -6.46397471e+02  3.28426563e+03  0.00000000e+00
  -1.12329282e+04 -8.75002266e-03]]
det(Jse)  =  -1.9803210101885482e+25


### Jhs

In [35]:
#dehel/dXSoI
hel_aelements=ray.conics[2]
hel_mu=hel_aelements[-2]
hel_elements=np.array(hel_aelements[:6])
hel=KeplerianOrbit(hel_mu)
hel.setElements(hel_elements,0)
hel.calcJacobians()
Jhs=np.copy(hel.Jkc)
print("Jhs := dehel/dXsoi = ",Jhs)
detJhs=np.linalg.det(Jhs)
print("det(Jse)  = ",detJhs)

Jhs := dehel/dXsoi =  [[ 7.69123606e-01  5.30370519e+00  2.02804080e-02 -2.60663355e+07
   1.50315370e+07 -4.68979746e+06]
 [-2.86052762e-12  6.74022613e-12 -6.19266682e-13 -3.61020347e-05
   3.89708478e-05 -6.86840134e-06]
 [-4.72051920e-13  5.77152713e-14  2.80869549e-12  5.00044691e-06
  -6.11377981e-07 -2.97525168e-05]
 [-6.75629975e-12  8.26056747e-13  4.01997913e-11 -6.79136142e-07
   8.30343549e-08  4.04084072e-06]
 [ 1.82812220e-11  9.88370457e-12 -3.79025747e-11 -1.19206390e-04
  -1.09418748e-05 -2.39082960e-05]
 [-4.54876607e-12 -1.22984193e-11 -5.11784618e-13  9.53774809e-05
  -4.19220830e-05  1.68913531e-05]]
det(Jse)  =  3.1212351298525724e-35


### Composed: Jhe = Jhs * Jse

In [36]:
detJhe=detJse*detJhs
print("Composed Jhe := dehel/degeosoi = ",detJhe)

Composed Jhe := dehel/degeosoi =  -6.181047505385631e-10


### Gobal standard

In [37]:
detJhi=detJei*detJhe
print("Total Jhi := dehel/dRimp = ",detJhi)

Total Jhi := dehel/dRimp =  -2.45858099081364e-13


### Comparison

In [38]:
print("Analytical Jhi := dehel/dRimp = ",detJhi)
print("Numerical Jhi := (dehel/dRimp)num = ",detJhi_num)

Analytical Jhi := dehel/dRimp =  -2.45858099081364e-13
Numerical Jhi := (dehel/dRimp)num =  7.926907170013695e-14


In conclusion: I get the order of magnitude but not the right value

### An interesting property

In [39]:
detJxi*detJex*detJse*detJhs

-2.45858099081364e-13

In [40]:
detJex*detJse

1.0000000000000355

This can be explained by the fact that the Jacobian |dx/de| does not depend on M.  Then when |dx/de| is evaluated in the surface it will give us the same value as |dx/de| at SoI.  Now, since |dx/de|=1/|de/dx|, then |dx/degeo||desoi/dx| = 1

## Analytical jacobians (alternative method)

We want to transform a vector Rimp: (lon,lat,alt,A,h,v) into (unbounded) orbital elements Ehel: (Q, E, I, O, W, m).

This process is complex and involves the following steps and corresponding Jacobians:

- Rimp to Xgeo: Jxi = dXgeo/dRimp
- Xgeo to Xsoi: Jsx = dXsoi/dXgeo
- Xsoi to ehel: Jhs = dehel/dXsoi

Here dXsoi/dXgeo correspond to the state transition matrix in the geocentric system.

The full jacobian is:

- |Jhs| |Jsx| |Jxi| = Jhi = dehel/dRimp

### Numerical state transition matrix: Jsx := dXsoi/dXgeo

In [41]:
def Xgeo2Xsoi(X,mu=1,tpos=0):
    global earth
    #Transform state into orbital elements
    orbitgeo=KeplerianOrbit(mu)
    orbitgeo.setState(X,0)
    #Propagate until Rhill
    a,e,i,W,w,M=orbitgeo.celements
    q=a*(1-e)
    n=np.sqrt(mu/np.abs(a)**3)
    fd=np.arccos((q*(1+e)/earth.rhill-1)/e)
    Hd=2*np.arctanh(np.sqrt((e-1)/(e+1))*np.tan(fd/2))
    Md=e*np.sinh(Hd)-Hd
    deltet=-Md/n
    #print(deltet)
    state=spy.conics([q,e,i,W,w,M,0,mu],-deltet)
    #print(state)
    #Get state at SoI
    orbitsoi=KeplerianOrbit(mu)
    orbitsoi.setElements([q,e,i,W,w,-Md],0)
    #print(f"By time: {state}, By M: {orbitsoi.state}")
    earth.updateBody(tpos)
    helstate=orbitsoi.state+earth.stateHelio
    #return helstate
    #return orbitsoi.state
    helstate=state+earth.stateHelio
    return state

In [42]:
X=geo.state
dX=np.abs(X*1e-6)
args=dict(mu=geo_mu,tpos=hel_aelements[-1])
Xsoi=Xgeo2Xsoi(X,**args)
print("Surface state = ",X)
print("SoI state = ",Xsoi)

Surface state =  [-3.56951589e+06  1.21037661e+06  5.15653310e+06  1.16402010e+03
  1.58121415e+04 -9.23101494e+03]
SoI state =  [ 3.39060624e+08  1.03391886e+09 -1.02498354e+09  3.34407144e+03
  1.00266669e+04 -1.00296820e+04]


In [43]:
Jsx=Util.computeJacobian(Xgeo2Xsoi,X,dX,N=6,**args)
print("State transition matrix: Jsx:= dXsoi/dXgeo = ",Jsx)
detJsx=np.linalg.det(Jsx)
print("det(Jsx) = ",detJsx)

State transition matrix: Jsx:= dXsoi/dXgeo =  [[-1.91414118e+01 -5.91196595e+01 -5.69628191e+01  7.73762879e+04
  -3.83419412e+04  7.46445650e+03]
 [-5.29396060e+01  5.56175497e+01  2.54520851e+01 -2.45872122e+04
   7.27955304e+04  3.85143366e+04]
 [-5.98416101e+01  3.53559693e+01  7.55917895e+00  1.05724555e+03
   6.05571040e+04  4.09862268e+04]
 [-2.83937997e-04 -5.47740806e-04 -4.31290090e-04  7.70288645e-01
  -1.25079775e-01 -7.25902250e-02]
 [-7.75277207e-04  6.23129343e-04  6.20366479e-04 -1.85341716e-01
   1.45986717e+00 -6.22346804e-02]
 [-3.25299152e-04  2.55726578e-04 -3.10089984e-04 -4.45190274e-02
  -1.58807837e-01  8.33954890e-01]]
det(Jsx) =  0.9999622342221709


### Gobal alternative

In [44]:
print(detJhs,detJsx,detJxi)

3.1212351298525724e-35 0.9999622342221709 -7.876948991438786e+21


In [45]:
detJhi_alt=detJhs*detJxi
print("Total Jhi (alternative) := dehel/dRimp = ",detJhi_alt)
print("Numerical Jhi := (dehel/dRimp)num = ",detJhi_num)

Total Jhi (alternative) := dehel/dRimp =  -2.458580990813553e-13
Numerical Jhi := (dehel/dRimp)num =  7.926907170013695e-14
